This file needs requires external data!

(Run on workstation)

In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt

import sys
sys.path.append('../src')

from alldata import AllData
from ensemble import EnsembleMember


np.set_printoptions(precision=2)
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

Input parameters

In [ ]:
#Choose starting year
year0 = 1951
#year0 = 1871

#Choose ensemble member
ism = 'CISM_NCA'
esm = 'EC-Earth3'
ssp = '245'

#Choose basal melt parameterisation
bmp = 'lin'

Computation

In [ ]:
#Read data
ad = AllData()
ad.year0 = year0
ad.gather()

#Compute iteration of ensemble member
ens = EnsembleMember(ad,ism=ism,esm=esm,ssp=ssp)
ens.iterate()

Figure settings

In [ ]:
plt.style.use('../src/style_paper')

mpl.rc('figure.subplot',left=.15)

savename = f'../draftfigs/single_ensemble_{ad.year0}{ad.option}.png'

Create and save figure

In [ ]:
fig,ax = plt.subplots(3,5,sharex=True,sharey='row')

#Choose variables to plot
varbs = [ens.TMP,.001*np.cumsum(ens.IML,axis=1),100*ens.SLR]

for b,bas in enumerate(ens.basin.values):
    for v,var in enumerate(varbs):
        dax = ax[v,b]
        dax.plot(ad.time,var[0,:,b],c='.5')
        for n in range(1,ens.TMP.shape[0]):
            dax.plot(ad.time,var[n,:,b],c=ad.bcol[bas])
        dax.text(.1,.8,f"{100*((var[-1,-1,b]-var[-1,0,b])-(var[0,-1,b]-var[0,0,b]))/np.abs(var[0,-1,b]-var[0,0,b]):+.0f}%",transform=dax.transAxes,weight='bold',c=ad.bcol[bas])
        
    ax[0,b].set_title(bas)
ax[-1,2].set_xlabel('Time [years]')
ax[0,0].set_ylabel('Temperature [degC]')
ax[1,0].set_ylabel('Cumulative ice mass loss \n [1000 Gt]')
ax[2,0].set_ylabel('Sea level rise [cm]')

plt.savefig(savename)
plt.show()